# Document translation batch (asynchronous mode)

https://learn.microsoft.com/en-us/azure/ai-services/translator/document-translation/how-to-guides/use-rest-api-programmatically?tabs=csharp

<img src="https://th.bing.com/th/id/OIP.ngRD8aNJwXZ2B0eol5tFlwAAAA?rs=1&pid=ImgDetMain" width=200>

In [1]:
import datetime
import json
import os
import requests
import sys
import time

from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.document import DocumentTranslationClient
from dotenv import load_dotenv

In [2]:
sys.version

'3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 10-Apr-2024 09:33:23


## Azure AI Translator credentials

In [4]:
load_dotenv("azure.env")

key = os.getenv("AZURE_AI_TRANSLATION_KEY")
endpoint = os.getenv("AZURE_AI_TRANSLATION_ENDPOINTDOCUMENT")
region = os.getenv("AZURE_AI_TRANSLATION_REGION")

## Blog storage access

In [5]:
sourceblobsas = "tobereplaced"
sourcesastoken = "tobereplaced"

In [6]:
targetblobsas = "tobereplaced"
targetsastoken = "tobereplaced"

In [7]:
sourceUri = sourceblobsas + sourcesastoken
targetUri = targetblobsas + targetsastoken

## Azure AI Translator client

In [8]:
# initialize a new instance of the DocumentTranslationClient object to interact with the Document Translation feature
client = DocumentTranslationClient(endpoint, AzureKeyCredential(key))

## Example

In [9]:
targetLanguage = "fr"

> https://aka.ms/TranslatorLanguageCodes 

In [11]:
# include source and target locations and target language code for the begin translation operation
print(
    f"Starting to translate the documents to language:{targetLanguage} ...")

start = time.time()

poller = client.begin_translation(sourceUri, targetUri, targetLanguage)
result = poller.result()

elapsed = time.time() - start
print("\nElapsed time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed %
      1)[2:])[:15], time.gmtime(elapsed)))

Starting to translate the documents to language:fr ...

Elapsed time: 00:01:49.153155


## Results

In [12]:
print("\033[1;31;34m")
print("Status: {}".format(poller.status()))
print("Created on: {}".format(poller.details.created_on))
print("Last updated on: {}".format(poller.details.last_updated_on))
print(
    "\nTotal number of translations on documents: {}".format(
        poller.details.documents_total_count
    )
)

print("  {} failed".format(poller.details.documents_failed_count))
print("  {} succeeded".format(poller.details.documents_succeeded_count))


Status: Succeeded
Created on: 2024-04-10 09:34:11.349999+00:00
Last updated on: 2024-04-10 09:35:46.138401+00:00

Total number of translations on documents: 4
  0 failed
  4 succeeded


In [13]:
nb = 1

for document in result:
    print("- Document", nb)
    print("Document ID: {}".format(document.id))
    print("Document status: {}".format(document.status))

    if document.status == "Succeeded":
        print("Source document location: {}".format(
            document.source_document_url))
        print(
            "Translated document location: {}".format(
                document.translated_document_url)
        )
        print("Translated to language: {}\n".format(document.translated_to))
        nb += 1

- Document 1
Document ID: 004dcbc9-0000-0000-0000-000000000000
Document status: Succeeded
Source document location: https://azurestorageaccountsr.blob.core.windows.net:443/docs/termsconditions_en.pdf
Translated document location: https://azurestorageaccountsr.blob.core.windows.net:443/docs-translated/termsconditions_en.pdf
Translated to language: fr

- Document 2
Document ID: 004dcbc8-0000-0000-0000-000000000000
Document status: Succeeded
Source document location: https://azurestorageaccountsr.blob.core.windows.net:443/docs/releasenotes_en.pdf
Translated document location: https://azurestorageaccountsr.blob.core.windows.net:443/docs-translated/releasenotes_en.pdf
Translated to language: fr

- Document 3
Document ID: 004dcbc7-0000-0000-0000-000000000000
Document status: Succeeded
Source document location: https://azurestorageaccountsr.blob.core.windows.net:443/docs/Microsoft%20Translator%20Customer%20Ready%20Deck%20v4.1.pptx
Translated document location: https://azurestorageaccountsr.bl